In [1]:
import torchvision.models as models
import torch.nn as nn
import torch
import loaders
import hyperparameters
import training

In [15]:
device = "cuda" if torch.cuda.is_available() else "cpu"
## setup current model and load weights
resnet50 = models.resnet50()
num_classes = 7
resnet50.fc = nn.Linear(resnet50.fc.in_features, num_classes)
pretrained = torch.load("./saved_model/resnet50_on_FER.pth")
resnet50.load_state_dict(pretrained["state_dict"])
resnet50.to(device)
resnet50.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [16]:
## make some evaluation on full image (no mask)
full_val_loader = loaders.get_loader(
    mask="full", train=False
)
# VAL_BATCH_SIZE = 100
# full_val_transform = transforms.Compose(
#     [
#         transforms.ToTensor(),
#         transforms.Normalize(
#             (0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)
#         )
#     ]
# )
# full_val_set = FacialImageData(
#     directory="./data/test",
#     transform=full_val_transform
# )
# full_val_loader = DataLoader(
#     full_val_set,
#     batch_size=VAL_BATCH_SIZE,
#     shuffle=False,
#     num_workers=2
# )

In [8]:
loss_func = nn.CrossEntropyLoss().to(device)

In [9]:
with torch.no_grad():
    val_loss = 0
    total_examples = 0
    correct_examples = 0
    for batch_idx, (inputs, targets) in enumerate(full_val_loader):
        # copy inputs to device
        inputs = inputs.to(device)
        targets = targets.to(device)
        # compute the output and loss
        out = resnet50(inputs)
        loss = loss_func(out, targets)
        # count the number of correctly predicted samples
        # in the current batch
        _, predicted = torch.max(out, 1)
        correct = predicted.eq(targets).sum()
        val_loss += loss.detach().cpu()
        total_examples += targets.shape[0]
        correct_examples += correct.item()
avg_loss = val_loss / len(full_val_loader)
avg_acc = correct_examples / total_examples
print(
    "Validation loss: %.4f, Validation accuracy: %.4f" % (avg_loss, avg_acc)
)

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Validation loss: 0.9739, Validation accuracy: 0.6649


In [10]:
## make some evaluation on upper masked image (upper mask)
upper_mask_val_loader = loaders.get_loader(
    mask="upper", train=False
)
# upper_mask_val_transform = transforms.Compose(
#     [
#         transforms.ToTensor(),
#         ImgMask([1, 2]),
#         transforms.Normalize(
#             (0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)
#         )
#     ]
# )
# upper_mask_val_set = FacialImageData(
#     directory="./data/test",
#     transform=upper_mask_val_transform
# )
# upper_mask_val_loader = DataLoader(
#     upper_mask_val_set,
#     batch_size=VAL_BATCH_SIZE,
#     shuffle=False,
#     num_workers=2
# )

In [11]:
with torch.no_grad():
    val_loss = 0
    total_examples = 0
    correct_examples = 0
    for batch_idx, (inputs, targets) in enumerate(upper_mask_val_loader):
        # copy inputs to device
        inputs = inputs.to(device)
        targets = targets.to(device)
        # compute the output and loss
        out = resnet50(inputs)
        loss = loss_func(out, targets)
        # count the number of correctly predicted samples
        # in the current batch
        _, predicted = torch.max(out, 1)
        correct = predicted.eq(targets).sum()
        val_loss += loss.detach().cpu()
        total_examples += targets.shape[0]
        correct_examples += correct.item()
avg_loss = val_loss / len(upper_mask_val_loader)
avg_acc = correct_examples / total_examples
print(
    "Validation loss: %.4f, Validation accuracy: %.4f" % (avg_loss, avg_acc)
)

Validation loss: 1.9198, Validation accuracy: 0.4489


In [ ]:
## further fine tune the model
## define hyperparameters


In [12]:
## trainning data
upper_mask_train_loader = loaders.get_loader(
    mask="upper", train=True
)
# TRAIN_BATCH_SIZE = 128
# upper_mask_train_transform = transforms.Compose(
#     [
#         # transforms.RandomCrop(48, padding=4),
#         # transforms.RandomHorizontalFlip(),
#         transforms.ToTensor(),
#         ImgMask([1, 2]),
#         transforms.Normalize(
#             (0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)
#         )
#     ]
# )
# upper_mask_train_set = FacialImageData(
#     directory="./data/test",
#     transform=upper_mask_train_transform
# )
# upper_mask_train_loader = DataLoader(
#     upper_mask_train_set,
#     batch_size=TRAIN_BATCH_SIZE,
#     shuffle=True,
#     num_workers=2
# )

In [17]:
training.train_model(
    resnet50,
    device,
    hyperparameters.CHECKPOINT_FOLDER,
    "upper.pth",
    hyperparameters.LR,
    hyperparameters.MOMENTUM,
    hyperparameters.REG,
    15,
    upper_mask_train_loader,
    upper_mask_val_loader,
    loss_func,
    full_val_loader,
    0.1,
    5
)

==> Training starts!
Epoch 0:
Training loss: 1.1685, Training accuracy: 0.5575
Validation loss: 1.1629, Validation accuracy: 0.5589
Saving ...

Validation loss: 1.0666, Validation accuracy: 0.6089
Epoch 1:
Training loss: 0.9843, Training accuracy: 0.6305
Validation loss: 1.1577, Validation accuracy: 0.5685
Saving ...

Validation loss: 1.1056, Validation accuracy: 0.5858
Epoch 2:
Training loss: 0.8511, Training accuracy: 0.6846
Validation loss: 1.2144, Validation accuracy: 0.5662

Validation loss: 1.1866, Validation accuracy: 0.5908
Epoch 3:
Training loss: 0.7400, Training accuracy: 0.7278
Validation loss: 1.3535, Validation accuracy: 0.5495

Validation loss: 1.3631, Validation accuracy: 0.5762
Epoch 4:
Training loss: 0.6459, Training accuracy: 0.7624
Validation loss: 1.3194, Validation accuracy: 0.5701
Saving ...

Validation loss: 1.2578, Validation accuracy: 0.5933
Epoch 5:
Training loss: 0.5452, Training accuracy: 0.8013
Validation loss: 1.3840, Validation accuracy: 0.5690

Validatio

([0.5575255146469749,
  0.6304991466090772,
  0.6846285137065032,
  0.7277508795151346,
  0.7624438329443728,
  0.8012818279981887,
  0.9179351422898743,
  0.9677104740673657,
  0.9834894980668083,
  0.9908391096868577,
  0.9940088473997701,
  0.9960639520707792,
  0.9960987843533387,
  0.9961684489184576,
  0.9972134173952419],
 [tensor(1.1685),
  tensor(0.9843),
  tensor(0.8511),
  tensor(0.7400),
  tensor(0.6459),
  tensor(0.5452),
  tensor(0.2486),
  tensor(0.1083),
  tensor(0.0580),
  tensor(0.0346),
  tensor(0.0235),
  tensor(0.0159),
  tensor(0.0154),
  tensor(0.0132),
  tensor(0.0117)],
 [0.5589300640847032,
  0.5685427695736974,
  0.5661744218445249,
  0.5494566731680134,
  0.5700752298690444,
  0.5689607132906102,
  0.6082474226804123,
  0.6042073000835887,
  0.6079687935358038,
  0.6023962106436334,
  0.601142379492895,
  0.6028141543605461,
  0.5998885483421565,
  0.6060183895235441,
  0.6018389523544163],
 [tensor(1.1629),
  tensor(1.1577),
  tensor(1.2144),
  tensor(1.353

In [ ]:
# ## create optimizer
# optimizer = optim.SGD(
#     resnet50.parameters(),
#     lr=hyperparameters.LR,
#     momentum=hyperparameters.MOMENTUM,
#     weight_decay=hyperparameters.REG
# )

# # the folder where the trained model is saved
# CHECKPOINT_FOLDER = "./saved_model"
# best_val_acc = 0
# resnet50 = resnet50.to(device)
# current_learning_rate = hyperparameters.LR


# train_loss_hist = []
# train_acc_hist = []
# test_loss_hist = []
# test_acc_hist = []
# # start the training/validation process
# print("==> Training starts!")
# print("="*50)
# for i in range(0, hyperparameters.NUM_EPOCH):
#     print("Epoch %d:" %i)
#     ## Train on the train set
#     #####################################################################
#     # switch to train mode
#     resnet50.train()
#     # this help you compute the training accuracy
#     total_examples = 0
#     correct_examples = 0
#     train_loss = 0 # track training loss if you want
#     # Train the model for 1 epoch.
#     for batch_idx, (inputs, targets) in enumerate(upper_mask_train_loader):
#         # copy inputs to device
#         inputs = inputs.to(device)
#         targets = targets.to(device)
#         # compute the output and loss
#         out = resnet50(inputs)
#         loss = loss_func(out, targets)
#         # zero the gradient
#         optimizer.zero_grad()
#         # backpropagation
#         loss.backward()
#         # apply gradient and update the weights
#         optimizer.step()
#         # count the number of correctly predicted samples in the current batch
#         _, predicted = torch.max(out, 1)
#         correct = predicted.eq(targets).sum()
#         train_loss += loss.detach().cpu()
#         total_examples += targets.shape[0]
#         correct_examples += correct.item()
#     avg_loss = train_loss / len(upper_mask_train_loader)
#     train_loss_hist.append(avg_loss)
#     avg_acc = correct_examples / total_examples
#     train_acc_hist.append(avg_acc)
#     print("Training loss: %.4f, Training accuracy: %.4f" %(avg_loss, avg_acc))
#     ######################################################################

#     # Validate on the validation dataset (masked)
#     ######################################################################
#     # switch to eval mode
#     resnet50.eval()
#     # this help you compute the validation accuracy
#     total_examples = 0
#     correct_examples = 0
#     val_loss = 0 # track the validation loss
#     # disable gradient during validation, which can save GPU memory
#     with torch.no_grad():
#         for batch_idx, (inputs, targets) in enumerate(upper_mask_val_loader):
#             # copy inputs to device
#             inputs = inputs.to(device)
#             targets = targets.to(device)
#             # compute the output and loss
#             out = resnet50(inputs)
#             loss = loss_func(out, targets)
#             # count the number of correctly predicted samples
#             # in the current batch
#             _, predicted = torch.max(out, 1)
#             correct = predicted.eq(targets).sum()
#             val_loss += loss.detach().cpu()
#             total_examples += targets.shape[0]
#             correct_examples += correct.item()
#     avg_loss = val_loss / len(upper_mask_val_loader)
#     test_loss_hist.append(avg_loss)
#     avg_acc = correct_examples / total_examples
#     test_acc_hist.append(avg_acc)
#     print(
#         "Validation loss: %.4f, Validation accuracy: %.4f" % (avg_loss, avg_acc)
#     )
#     ######################################################################

#     # save the model checkpoint
#     if avg_acc > best_val_acc:
#         best_val_acc = avg_acc
#         if not os.path.exists(CHECKPOINT_FOLDER):
#            os.makedirs(CHECKPOINT_FOLDER)
#         print("Saving ...")
#         state = {'state_dict': resnet50.state_dict(),
#                 'epoch': i,
#                 'lr': current_learning_rate}
#         torch.save(state, os.path.join(CHECKPOINT_FOLDER, 'resnet50_on_upper_masked.pth'))
#     print('')
#     ## decay learning rate
#     # if i % DECAY_EPOCH == 0 and i != 0:
#     #     current_learning_rate *= LR_DECAY
#     #     for param_group in optimizer.param_groups:
#     #         param_group["lr"] = current_learning_rate
#     #         print(f"Current learning rate has decayed to %f" %current_learning_rate)

# print("="*50)
# print(
#     f"==> Optimization finished! Best validation accuracy: {best_val_acc:.4f}"
# )